In [ ]:
import os
import json
import pandas as pd
from datetime import date
from medcat.cat import CAT

In [ ]:
# will be used to date the trained model
today = str(date.today())
today = today.replace("-","")

In [ ]:
ann_dir = "working_with_cogstack/data/annotated_docs/"
mctrainer_export_path = ann_dir + "MedCAT_Export_With_Text_2021-08-25_19_55_45.json"  # name of your mct export

model_dir = 'working_with_cogstack/models/modelpack'

modelpack = '' # name of modelpack
model_pack_path = os.path.join(model_dir, modelpack)

output_modelpack = model_dir + f"{today}_trained_model"

# Add training filter if needed
snomed_filter_path = None  # path to snomed filter

In [ ]:
# Create CAT - the main class from medcat used for concept annotation
cat = CAT.load_model_pack(model_pack_path)
cat.config.linking['filters'] = {'cuis':set()}  # To remove exisitng filters

# Set filter

This will speed up the training time. As you will only train a select number of concepts at once.

In [ ]:
# Add extra training filter if required.
if snomed_filter_path:
    snomed_filter = set(json.load(open(snomed_filter_path)))
else:
    snomed_filter = set(cat.cdb.cui2preferred_name.keys())


# Train

In [ ]:
cat.train_supervised_from_json(data_path=mctrainer_export_path, 
                               nepochs=3,
                               reset_cui_count=False,
                               print_stats=True,
                               use_filters=True,
                               extra_cui_filter=snomed_filter,  # If not filter is set remove this line
                               )


# Stats

In [ ]:
data = json.load(open(mctrainer_export_path))

In [ ]:
fps, fns, tps, cui_prec, cui_rec, cui_f1, cui_counts, examples = cat._print_stats(data, use_filters=True)

In [ ]:
cui = "22298006" # Myocardial infarction
print(cui_f1[cui], cui_prec[cui], cui_rec[cui])

# Save

Also remember that you can save specific components within the modelpack. Rather than create a new one

In [ ]:
# save modelpack
cat.create_model_pack(os.path.join(model_dir, output_modelpack))

# Test

In [ ]:
text = "The pateint has hypertension and an MI"
doc = cat.get_entities(text)

In [ ]:
doc